In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

**Table of contents**<a id='toc0_'></a>    
- [ Llamadas básicas al modelo](#toc1_)    
- [Creación y ejecución del modelo usando LangChain](#toc2_)    
- [ Creación de un prompt parametrizado](#toc3_)    
- [Estructuración de la salida](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ Llamadas básicas al modelo](#toc0_)

In [3]:
import google.generativeai as genai
import os

# Configura el modelo utilizando la clave de API almacenada en el entorno
genai.configure(api_key = os.getenv("GOOGLE_API_KEY"))

# Función que realiza una llamada directa al modelo gemini-2.5-flash
def get_completion(prompt, model = "gemini-2.5-flash"):
    
    # Se crea una instancia del modelo generativo especificado
    chat_model = genai.GenerativeModel(model)
    
    # Se genera una respuesta con temperatura 0 para minimizar la aleatoriedad
    response = chat_model.generate_content(
        prompt,
        generation_config = {"temperature": 0}
    )
    
    # Se retorna únicamente el texto de la respuesta
    return response.text

In [4]:
# Se obtiene la respuesta del modelo a partir de un prompt simple
response = get_completion("What is the capital of Colombia?")  

# Se imprime la interacción de forma simulada entre el usuario y el modelo
print("\nUser: What is the capital of Colombia?")  
print("Model: " + response)

E0000 00:00:1760848545.959295  153622 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



User: What is the capital of Colombia?
Model: The capital of Colombia is **Bogotá**.


# <a id='toc2_'></a>[Creación y ejecución del modelo usando LangChain](#toc0_)

In [5]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
# Se crea una instancia del modelo gemini-2.5-flash con temperatura 0 (respuestas determinísticas)
chat = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",     
    temperature=0.0             
)

E0000 00:00:1760848550.306096  153622 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [6]:
# Se define el mensaje que se enviará al modelo, incluyendo la instrucción, el contenido del mensaje y el formato esperado
prompt_customer_1 = """
Analyze the following message and classify it according to its overall sentiment and level of urgency.

Message: ```I placed an order over a week ago and received a product that was completely different from what I purchased. I have sent three emails to customer support and still have not received a response.
This is unacceptable. I need an urgent solution or I will escalate the case to the superintendent.```

Return your response in the following format:
sentiment: <positive | neutral | negative>
urgency: <high | medium | low>
"""

# Se envía el mensaje al modelo como un objeto HumanMessage y se obtiene la respuesta
response = chat.invoke([HumanMessage(content = prompt_customer_1)])

# Se imprime únicamente el contenido textual de la respuesta generada por el modelo
print("\nClassification:\n")
print(response.content)


Classification:

sentiment: negative
urgency: high


In [7]:
# Se define el mensaje que se enviará al modelo, incluyendo la instrucción, el contenido del mensaje y el formato esperado
prompt_customer_2 = """
Analyze the following message and classify it according to its overall sentiment and level of urgency.

Message: ```Hello! I wanted to congratulate you on the improvements to the website; it is much faster and clearer. Just as a suggestion, you could consider adding a price filter in
the products section. It's not urgent, just an idea that I think could be useful.```

Return your response in the following format:
sentiment: <positive | neutral | negative>
urgency: <high | medium | low>
"""

# Se envía el mensaje al modelo como un objeto HumanMessage y se obtiene la respuesta
response = chat.invoke([HumanMessage(content = prompt_customer_2)])  

# Se imprime únicamente el contenido textual de la respuesta generada por el modelo
print("\nClassification:\n")
print(response.content)


Classification:

sentiment: positive
urgency: low


# <a id='toc3_'></a>[ Creación de un prompt parametrizado](#toc0_)

In [12]:
from langchain.prompts import ChatPromptTemplate

# Se define una plantilla de prompt con un marcador de posición {message}, que será reemplazado dinámicamente
template = """
Analyze the following message and classify it according to its overall sentiment and level of urgency.

Message: ```{message}```

Return your response in the following format:
sentiment: <positive | neutral | negative>
urgency: <high | medium | low>
"""

# Se construye un objeto ChatPromptTemplate a partir del texto con marcador de posición
prompt_template = ChatPromptTemplate.from_template(template)

In [9]:
# Se imprime la lista de variables de entrada definidas en la plantilla del prompt
print("\nInput variables for the classification prompt:\n")
print(prompt_template.input_variables)


Input variables for the classification prompt:

['message']


In [10]:
# Se define el primer mensaje de un cliente con tono negativo y urgencia alta
customer_1 = """I placed an order over a week ago and received a product that was completely different from what I purchased. I have sent three emails to customer support and still have not received a response.
This is unacceptable. I need an urgent solution or I will escalate the case to the superintendent."""

# Se genera el prompt final a partir del template, insertando el mensaje como valor del campo 'message'
messages = prompt_template.format_messages(message = customer_1)  

# Se envía el prompt generado al modelo y se obtiene la respuesta
response = chat(messages)

# Se imprime el contenido de la respuesta del modelo
print("\nClassification:\n")
print(response.content)

/tmp/ipykernel_153622/3280055889.py:9: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)



Classification:

sentiment: negative
urgency: high


In [13]:
# Se define el segundo mensaje de un cliente con tono positivo y una sugerencia sin urgencia
customer_2 = """Hello! I wanted to congratulate you on the improvements to the website; it is much faster and clearer. Just as a suggestion, you could consider adding a price filter in
the products section. It's not urgent, just an idea that I think could be useful."""

# Se genera el prompt final a partir del template, insertando el mensaje como valor del campo 'message'
messages = prompt_template.format_messages(message = customer_2) 

# Se envía el prompt generado al modelo y se obtiene la respuesta
response = chat(messages)

# Se imprime el contenido de la respuesta del modelo
print("\nClassification:\n")
print(response.content)


Classification:

sentiment: positive
urgency: low


# <a id='toc4_'></a>[Estructuración de la salida](#toc0_)

In [14]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Se definen los campos que se esperan en la salida del modelo,
# cada uno con su nombre y descripción
schemas = [
    ResponseSchema(name = "sentiment", description = "Message sentiment"),  # Sentimiento general del mensaje
    ResponseSchema(name = "urgency", description = "Message urgency level")  # Nivel de urgencia del mensaje
]

# Se construye un parser estructurado a partir del esquema definido
parser = StructuredOutputParser.from_response_schemas(schemas)

# Se genera la instrucción que se debe incluir en el prompt para que el modelo
# devuelva una respuesta que cumpla con el formato esperado por el parser
format_instructions = parser.get_format_instructions()

# Se imprime la instrucción de formato que debe incluirse dentro del prompt
print("\nInstructions generated by the parser:\n")
print(format_instructions)


Instructions generated by the parser:

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // Message sentiment
	"urgency": string  // Message urgency level
}
```


In [15]:
# Se define una nueva plantilla de prompt que incluye:
# 1. La instrucción general de clasificación.
# 2. El mensaje a analizar (variable dinámica: {message}).
# 3. Las instrucciones generadas por el parser para estructurar la salida (variable: {format_instructions}).

new_template = """
Analyze the following message and classify it according to its overall sentiment <positive | neutral | negative> and level of urgency <high | medium | low>. 

Message: ```{message}```

{format_instructions}
"""

In [16]:
# Se define el mensaje del cliente a analizar
customer_1 = """I placed an order over a week ago and received a product that was completely different from what I purchased. I have sent three emails to customer support and still have not received a response.
This is unacceptable. I need an urgent solution or I will escalate the case to the superintendent."""

# Se construye el template dinámico a partir del nuevo prompt con instrucciones estructurales
new_prompt_template = ChatPromptTemplate.from_template(new_template)

# Se genera el prompt final inyectando el mensaje y las instrucciones del parser
messages = new_prompt_template.format_messages(
    message = customer_1, 
    format_instructions = format_instructions 
)

# Se envía el mensaje al modelo y se obtiene la respuesta
response = chat(messages)

# Se utiliza el parser para convertir la respuesta del modelo en un diccionario estructurado
parsed = parser.parse(response.content)

# Se imprime el tipo del contenido original de la respuesta (sigue siendo str)
print("\nResponse content type (raw):\n")
print(type(response.content))  # <class 'str'>

# Se imprime el tipo del objeto ya parseado (estructurado)
print("\nParsed object type:\n")
print(type(parsed))  # <class 'dict'>

# Ahora es posible acceder a los valores mediante sus claves
print('\nValue of "urgency" key:\n')
print(parsed['urgency'])  # Ejemplo: 'high'


Response content type (raw):

<class 'str'>

Parsed object type:

<class 'dict'>

Value of "urgency" key:

high
